In [1]:
from __future__ import print_function # note --> this import forces one to use print() as in Python 3
                                      # resulting in code that can run in Python 2 or Python 3
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import flopy as fp
import pandas as pd
import os
from scipy.stats.mstats import gmean as sp_gmean
import matplotlib.pyplot as plt
%matplotlib inline

## Read in the "true" k array

In [2]:
inK = np.loadtxt('k-true.dat')
inK = np.log10(inK)

In [3]:
# find the geometric mean about which to stretch
inK_geomean = np.mean(inK)
inK_geomean

-0.46187011088715513

#### calculate stretching factors as deviations from the geometric mean on inK (the input K)

In [4]:
stretch_f = inK-inK_geomean

## Read in the head observation locations

In [5]:
obslox = pd.read_csv('hob.dat', skiprows=3, delim_whitespace=True, header=None)
r_obs = obslox[2]
c_obs = obslox[3]
name_obs = obslox[0]

#### Get the locations of the cell centers

In [6]:
inmod = fp.modflow.Modflow.load('Ex_forward_base.nam')
xy=inmod.dis.get_node_coordinates()
x_mod = xy[1]
y_mod = xy[0]

C:\Users\kniffin\AppData\Local\Continuum\Anaconda\lib\site-packages\flopy\modflow\mfdis.py:164: UserWarning: ModflowDis.sr is deprecated. use Modflow.sr
  warnings.warn("ModflowDis.sr is deprecated. use Modflow.sr")


#### Now find the X and Y locations of the observations

In [7]:
x_obs = [x_mod[i-1] for i in c_obs]
y_obs = [y_mod[i-1] for i in r_obs]

### Set up HYDMOD data

In [8]:
NHYD = len(name_obs)
IHYDUN = 424
HYDNOH = -9999999
PCKG = 'BAS'
ARR = 'HD'
hydinfo = []
INTYP = 'C'
for cob, cx, cy in zip(name_obs,x_obs,y_obs):
    hydinfo.append([PCKG, ARR, INTYP, 1, cx, cy, cob])

    

### Let's set a few values by which to stretch the K fields

In [9]:
amping_values = [0.001,0.01,0.1,0.5,1,2,3]

### Next we make a folder for each, create the reference model in there, and run to obtain observations

In [10]:
for c_amp in amping_values:
    # read in the base model
    cmod = fp.modflow.Modflow.load('Ex_forward_base.nam')
    # change the namefile and working directory - if the working directory doesn't exists, make it
    cmod.name = 'TrueK_amped_by_{0}'.format(c_amp)
    cmod.exe_name = 'mf2005'
    c_ws = 'TrueK_amped_by_{0}'.format(c_amp)
    if not os.path.exists(c_ws):
        os.mkdir(c_ws)
    cmod.model_ws = c_ws
    
    
    # stretch the K field and assign it to the lpf package
    curr_K = inK_geomean + stretch_f * c_amp
    cmod.lpf.hk[0] = 10**curr_K
    
    # make and save a quick plot of the new field as well
    plt.figure(figsize=(8,6))
    ax=plt.subplot(121)
    im=plt.imshow(curr_K, cmap='viridis', interpolation='nearest')
    plt.title('log10 K')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    plt.colorbar(im, cax=cax)

    ax=plt.subplot(122)
    im=plt.imshow(10**curr_K, cmap='viridis', interpolation='nearest')
    plt.title('K')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)
    
    plt.savefig(os.path.join(c_ws,'k_field{0}.pdf'.format(c_ws)))
    plt.close('all')
    
    # add the HYDMOD information
    hyd = fp.modflow.ModflowHyd(cmod, nhyd=NHYD, ihydun=IHYDUN, hydnoh=HYDNOH, obsdata=hydinfo)

    # fiddle with the PCG settings a bit to handle the really heterogeneous cases
    cmod.pcg.hclose = 1e-4
    cmod.pcg.relax = 0.7
    cmod.pcg.damp = 0.5
    cmod.pcg.mxiter = int(5e4)
    # now run the model
    cmod.write_input()
    cmod.run_model()


changing model workspace...
   TrueK_amped_by_0.001


AttributeError: 'NoneType' object has no attribute 'hk'

### Now we read in the HYDMOD Head Data and corrupt and save it in each folder

In [22]:
for c_amp in amping_values:
    # read in the hydmod data
    c_ws = 'TrueK_amped_by_{0}'.format(c_amp)
    hyd_file = os.path.join(c_ws,'{0}.hyd.bin'.format(c_ws))
    
    obs = fp.utils.HydmodObs(hyd_file)
    allobs = obs.get_dataframe(totim=obs.get_times()[-1]).T
    # doing some manipulations to the dataframe we read in from hydmod
    allobs.drop('totim', inplace=True)
    allobs['names'] = [i.replace('HDC001','') for i in allobs.index]
    allobs.index = allobs.names
    allobs.drop('names', axis=1, inplace=True)
    allobs.columns = ['obs']
    allobs['obs_0.1_std_noise'] = allobs.obs + (np.random.randn(len(allobs))*0.1)
    allobs.to_csv(os.path.join(c_ws,'observations.txt'), sep='\t')


IOError: [Errno 2] No such file or directory: 'TrueK_amped_by_0.001\\TrueK_amped_by_0.001.hyd.bin'

In [15]:
fp.__version__

'3.2.5'